In [ ]:
PAT = '<enter_info_from_your_account>'
USER_ID = 'clarifai'
APP_ID = 'main'
MODEL_ID = 'moderation-english-text-classification'
MODEL_VERSION_ID = '997336df717d4295a1f58fb28cb3cce2'


from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_code_pb2

channel = ClarifaiChannel.get_grpc_channel()
stub = service_pb2_grpc.V2Stub(channel)

metadata = (('authorization', 'Key ' + PAT),)

userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)


import pickle

with open('./data/comments.pkl', 'rb') as handle:
    comments = pickle.load(handle)

comments = list(set(comments))

from tqdm import tqdm

clarifai_moderation_map = {}

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

for chunk in tqdm(chunker(comments,128)):
    inputs = []
    for RAW_TEXT in chunk:
        inputs += [
                    resources_pb2.Input(
                        data=resources_pb2.Data(
                            text=resources_pb2.Text(
                                raw=RAW_TEXT
                            )
                        )
                    )
                ]

    post_model_outputs_response = ''
    while post_model_outputs_response == '':

        try:
            post_model_outputs_response = stub.PostModelOutputs(
                service_pb2.PostModelOutputsRequest(
                    user_app_id=userDataObject,  # The userDataObject is created in the overview and is required when using a PAT
                    model_id=MODEL_ID,
                    version_id=MODEL_VERSION_ID,  # This is optional. Defaults to the latest model version
                    inputs=inputs
                ),
                metadata=metadata
            )
            if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
                print(post_model_outputs_response.status)
                raise Exception("Post model outputs failed, status: " + post_model_outputs_response.status.description)
                
        except Exception as e:
            print(e)
            post_model_outputs_response = ''


    outputs = post_model_outputs_response.outputs

    results = []
    for comment, output in zip(chunk, outputs):
        block = {}
        for concept in output.data.concepts:
            block[concept.name] = concept.value

        clarifai_moderation_map[comment] = block

In [ ]:
with open('./results/comment_moderation_clarifai.pkl', 'wb') as handle:
    pickle.dump(clarifai_moderation_map, handle, protocol=pickle.HIGHEST_PROTOCOL)